## Standard Imports

In [2]:
import ee
import os
from pathlib import Path
import requests
from PIL import Image
import numpy as np
from io import BytesIO


print('Imported')

Imported


In [3]:

# Initialize Earth Engine
try:
    ee.Initialize(project = 'glacier-probe-model-475519')
except:
    ee.Authenticate()
    ee.Initialize(project = 'glacier-probe-model-475519')

print('Done')

Done


## 1. Hasdeo Forest RGB Downloader

In [3]:
"""
Hasdeo Forest GeoTIFF Downloader - MODEL TRAINING VERSION (1000+ Images)
=========================================================================
Downloads high-quality multi-band GeoTIFF satellite imagery from Sentinel-2
for the Hasdeo Arand forest region, covering full deforestation timeline.

TARGET: 1000+ unique image exports via:
  - Monthly composites across ALL years (2013–2025) × 7 regions  = 1,092
  - Annual mosaics (2013–2025) × 7 regions                       =    91
  - Dry-season composites (2013–2024) × 7 regions                =    84
  - Wet-season composites (2013–2024) × 7 regions                =    84
  TOTAL POTENTIAL                                                 = 1,351+

BANDS DOWNLOADED:
  - B2 (Blue, 490nm), B3 (Green, 560nm), B4 (Red, 665nm)
  - B8 (NIR, 842nm)       — critical for NDVI / vegetation indices
  - B8A (NIR narrow, 865nm)
  - B11 (SWIR1, 1610nm), B12 (SWIR2, 2190nm) — burn/soil detection
  - SCL (Scene Classification Layer)           — for masking
  - NDVI, NDWI, NBR, EVI  — pre-computed indices

OUTPUT:
  - Full-resolution Cloud-Optimized GeoTIFF (10m) with all bands
  - Cloud-masked monthly/seasonal/annual composites
  - Metadata CSV for dataset bookkeeping
  - Task ID JSON for GEE monitoring

USAGE:
  pip install earthengine-api google-cloud-storage requests tqdm
  python hasdeo_geotiff_downloader_1000.py
"""

import ee
import os
import csv
import time
import json
import requests
from pathlib import Path
from datetime import datetime
from tqdm import tqdm

# ─────────────────────────────────────────────────────────────────────────────
# CONFIGURATION
# ─────────────────────────────────────────────────────────────────────────────

PROJECT_ID    = 'glacier-probe-model-475519'   # ← your GEE project ID
GCS_BUCKET    = ''                             # ← your GCS bucket (for gcs mode)
OUTPUT_DIR    = 'hasdeo_training_dataset'

# ── 7 Regions covering Hasdeo Arand + sub-blocks ──────────────────────────
# Each region is [west, south, east, north] in WGS84
REGIONS = {
    # Full Hasdeo Arand forest block
    'Hasdeo_Full':       [82.60, 22.80, 83.00, 23.20],
    # PEKB (Parsa East & Kante Basan) — primary deforestation zone
    'PEKB_Core':         [82.70, 22.90, 82.85, 23.05],
    # Kente Extension block — active 2023-2026
    'Kente_Extension':   [82.65, 23.05, 82.80, 23.20],
    # Buffer / control area (less disturbed forest)
    'Control_Forest':    [82.85, 23.00, 83.00, 23.15],
    # Northern Hasdeo block
    'Hasdeo_North':      [82.60, 23.10, 82.80, 23.30],
    # Southern Hasdeo block
    'Hasdeo_South':      [82.70, 22.70, 82.90, 22.90],
    # Mining buffer / transition zone
    'Mining_Buffer':     [82.75, 22.85, 82.95, 23.05],
}

# ── Sentinel-2 band selection ──────────────────────────────────────────────
S2_BANDS   = ['B2', 'B3', 'B4', 'B8', 'B8A', 'B11', 'B12', 'SCL']
SCALE_METERS      = 10     # 10 m native resolution
MAX_CLOUD_COVER   = 20     # % per scene filter
MIN_IMAGES_PER_PERIOD = 1  # skip period if fewer scenes available

# ── Export mode ───────────────────────────────────────────────────────────
#   'drive'       → Google Drive / Hasdeo_Training_Dataset  (recommended)
#   'gcs'         → GCS bucket (fastest bulk; set GCS_BUCKET above)
#   'local_thumb' → small PNGs locally (no quota; QA only, not for training)
EXPORT_MODE = 'drive'

# ─────────────────────────────────────────────────────────────────────────────
# TIME PERIOD BUILDER  →  monthly for ALL years
# ─────────────────────────────────────────────────────────────────────────────

def build_monthly_periods():
    """
    Build monthly time windows from Jan 2013 to Aug 2025.
    12 months × 13 years × 7 regions = 1,092 potential composites.
    Labels reflect the deforestation phase for dataset annotation.
    """
    phase_map = {
        range(2013, 2016): 'initial_clearing',
        range(2016, 2020): 'rapid_expansion',
        range(2020, 2023): 'slowdown',
        range(2023, 2026): 'renewed_surge',
    }

    def get_phase(year):
        for yr_range, label in phase_map.items():
            if year in yr_range:
                return label
        return 'unknown'

    periods = []
    for year in range(2013, 2026):
        for month in range(1, 13):
            # Stop at August 2025 (data availability boundary)
            if year == 2025 and month > 8:
                break
            start = f'{year}-{month:02d}-01'
            if month == 12:
                end = f'{year + 1}-01-01'
            else:
                end = f'{year}-{month + 1:02d}-01'
            periods.append((start, end, get_phase(year)))
    return periods

TIME_PERIODS = build_monthly_periods()

# ─────────────────────────────────────────────────────────────────────────────
# INITIALIZATION
# ─────────────────────────────────────────────────────────────────────────────

def init_ee():
    try:
        ee.Initialize(project=PROJECT_ID)
        print(f"✓ Earth Engine initialized | project: {PROJECT_ID}")
    except Exception:
        print("⚠  Re-authenticating...")
        ee.Authenticate()
        ee.Initialize(project=PROJECT_ID)
        print("✓ Authenticated and initialized.")


def create_dirs():
    for sub in ['exports', 'metadata', 'composites', 'logs', 'annual', 'seasonal']:
        Path(f"{OUTPUT_DIR}/{sub}").mkdir(parents=True, exist_ok=True)
    print(f"✓ Output directories ready under: {OUTPUT_DIR}/")

# ─────────────────────────────────────────────────────────────────────────────
# CLOUD MASKING & INDEX COMPUTATION
# ─────────────────────────────────────────────────────────────────────────────

def mask_s2_clouds(image):
    """
    Dual-layer cloud mask:
      1. QA60 bitmask  — opaque cloud + cirrus
      2. SCL layer     — shadow (3), medium cloud (8), high cloud (9),
                         cirrus (10), snow/ice (11)
    """
    qa = image.select('QA60')
    qa_mask = (qa.bitwiseAnd(1 << 10).eq(0)
                 .And(qa.bitwiseAnd(1 << 11).eq(0)))

    scl = image.select('SCL')
    scl_mask = (scl.neq(3)
                   .And(scl.neq(8))
                   .And(scl.neq(9))
                   .And(scl.neq(10))
                   .And(scl.neq(11)))

    return image.updateMask(qa_mask.And(scl_mask))


def add_indices(image):
    """Add vegetation / burn / water indices useful for deforestation modelling."""
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    ndwi = image.normalizedDifference(['B3', 'B8']).rename('NDWI')
    nbr  = image.normalizedDifference(['B8', 'B12']).rename('NBR')
    evi  = image.expression(
        '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))',
        {'NIR':  image.select('B8'),
         'RED':  image.select('B4'),
         'BLUE': image.select('B2')}
    ).rename('EVI')
    return image.addBands([ndvi, ndwi, nbr, evi])


def get_s2_collection(roi, start_date, end_date):
    """Return cloud-filtered, index-enriched Sentinel-2 SR collection."""
    return (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
              .filterBounds(roi)
              .filterDate(start_date, end_date)
              .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', MAX_CLOUD_COVER))
              .select(S2_BANDS + ['QA60'])
              .map(mask_s2_clouds)
              .map(add_indices))

# ─────────────────────────────────────────────────────────────────────────────
# EXPORT FUNCTIONS
# ─────────────────────────────────────────────────────────────────────────────

def export_to_drive(image, description, region, scale=SCALE_METERS):
    """
    Export a full-resolution Cloud-Optimized GeoTIFF to Google Drive.
    Files appear in: My Drive → Hasdeo_Training_Dataset/
    """
    task = ee.batch.Export.image.toDrive(
        image          = image,
        description    = description,
        folder         = 'Hasdeo_Training_Dataset',
        fileNamePrefix = description,
        region         = region,
        scale          = scale,
        crs            = 'EPSG:32644',      # UTM Zone 44N — Hasdeo region
        fileFormat     = 'GeoTIFF',
        maxPixels      = 1e10,
        formatOptions  = {'cloudOptimized': True}
    )
    task.start()
    return task


def export_to_gcs(image, description, region, bucket, scale=SCALE_METERS):
    """Export to a Google Cloud Storage bucket."""
    task = ee.batch.Export.image.toCloudStorage(
        image          = image,
        description    = description,
        bucket         = bucket,
        fileNamePrefix = f'hasdeo/{description}',
        region         = region,
        scale          = scale,
        crs            = 'EPSG:32644',
        fileFormat     = 'GeoTIFF',
        maxPixels      = 1e10,
        formatOptions  = {'cloudOptimized': True}
    )
    task.start()
    return task


def download_local_thumb(image, region, filepath, bands=None, size=512):
    """
    Download a small RGB PNG for visual QA only.
    NOT suitable for model training — use drive/gcs modes for full res.
    """
    if bands is None:
        bands = ['B4', 'B3', 'B2']
    try:
        vis = image.visualize(bands=bands, min=0, max=3000)
        url = vis.getThumbURL({'region': region, 'dimensions': size, 'format': 'png'})
        r = requests.get(url, timeout=120)
        if r.status_code == 200:
            with open(filepath, 'wb') as f:
                f.write(r.content)
            return True
    except Exception as e:
        print(f"    ✗ Thumb error: {str(e)[:70]}")
    return False

# ─────────────────────────────────────────────────────────────────────────────
# COMPOSITE HELPERS
# ─────────────────────────────────────────────────────────────────────────────

def create_seasonal_composite(roi, year, season='dry'):
    """
    Median composite for a season:
      dry (Nov–Apr) — clearest skies, best for deforestation detection
      wet (May–Oct) — captures phenological changes
    """
    if season == 'dry':
        start, end = f'{year}-11-01', f'{year + 1}-04-30'
    else:
        start, end = f'{year}-05-01', f'{year}-10-31'

    col = get_s2_collection(roi, start, end)
    count = col.size().getInfo()
    composite = col.median().clip(roi)
    return composite, count


def create_annual_mosaic(roi, year):
    """Best-pixel (median) annual mosaic for a full calendar year."""
    col = get_s2_collection(roi, f'{year}-01-01', f'{year}-12-31')
    count = col.size().getInfo()
    mosaic = col.median().clip(roi)
    return mosaic, count

# ─────────────────────────────────────────────────────────────────────────────
# METADATA TRACKING
# ─────────────────────────────────────────────────────────────────────────────

metadata_records = []

def log_export(task_id, description, region_name,
               start_date, end_date, image_count, phase, export_type):
    metadata_records.append({
        'task_id':     task_id,
        'description': description,
        'region':      region_name,
        'start_date':  start_date,
        'end_date':    end_date,
        'image_count': image_count,
        'phase':       phase,
        'export_type': export_type,
        'timestamp':   datetime.now().isoformat(),
        'scale_m':     SCALE_METERS,
        'crs':         'EPSG:32644',
        'bands':       ','.join(S2_BANDS + ['NDVI', 'NDWI', 'NBR', 'EVI']),
    })


def save_metadata_csv():
    if not metadata_records:
        return
    csv_path = f"{OUTPUT_DIR}/metadata/export_log.csv"
    with open(csv_path, 'w', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=metadata_records[0].keys())
        writer.writeheader()
        writer.writerows(metadata_records)
    print(f"✓ Metadata saved: {csv_path}  ({len(metadata_records)} records)")


def save_task_ids(tasks):
    """Persist all GEE Export task IDs for later monitoring."""
    task_file = f"{OUTPUT_DIR}/logs/task_ids.json"
    data = []
    for t in tasks:
        if t is None:
            continue
        try:
            data.append({
                'id':          t.id,
                'description': t.config.get('description', ''),
                'status':      t.status().get('state', 'UNKNOWN'),
            })
        except Exception:
            pass
    with open(task_file, 'w') as f:
        json.dump(data, f, indent=2)
    print(f"✓ Task IDs saved: {task_file}")
    print(f"  ▶  Monitor: https://code.earthengine.google.com/tasks")

# ─────────────────────────────────────────────────────────────────────────────
# GENERIC EXPORT DISPATCHER
# ─────────────────────────────────────────────────────────────────────────────

def dispatch_export(composite, desc, roi,
                    region_name, start_date, end_date,
                    count, phase, all_tasks,
                    thumb_subdir='exports'):
    """
    Route the export to the configured backend and log metadata.
    Returns True on success, False on skip/error.
    """
    task = None
    try:
        if EXPORT_MODE == 'drive':
            task = export_to_drive(composite, desc, roi)
            all_tasks.append(task)

        elif EXPORT_MODE == 'gcs':
            if not GCS_BUCKET:
                raise ValueError("Set GCS_BUCKET in config for 'gcs' mode")
            task = export_to_gcs(composite, desc, roi, GCS_BUCKET)
            all_tasks.append(task)

        else:  # local_thumb
            thumb_path = f"{OUTPUT_DIR}/{thumb_subdir}/{desc}.png"
            if os.path.exists(thumb_path):
                return False  # already downloaded
            success = download_local_thumb(composite, roi, thumb_path)
            if not success:
                return False

        task_id = task.id if task else 'local'
        log_export(task_id, desc, region_name,
                   start_date, end_date, count, phase, EXPORT_MODE)
        return True

    except Exception as e:
        print(f"    ✗ Export error [{desc}]: {str(e)[:80]}")
        return False

# ─────────────────────────────────────────────────────────────────────────────
# MAIN
# ─────────────────────────────────────────────────────────────────────────────

def main():
    # ── Header ──────────────────────────────────────────────────────────────
    monthly_potential  = len(TIME_PERIODS) * len(REGIONS)
    annual_potential   = 13 * len(REGIONS)           # 2013-2025
    seasonal_potential = 12 * len(REGIONS) * 2       # dry + wet, 2013-2024
    total_potential    = monthly_potential + annual_potential + seasonal_potential

    print("=" * 72)
    print("  HASDEO FOREST — GeoTIFF DOWNLOADER v2  (TARGET: 1 000+ IMAGES)")
    print("=" * 72)
    print(f"  Export mode      : {EXPORT_MODE.upper()}")
    print(f"  Resolution       : {SCALE_METERS} m")
    print(f"  Max cloud cover  : {MAX_CLOUD_COVER}%")
    print(f"  Regions          : {len(REGIONS)}  → {list(REGIONS.keys())}")
    print(f"  Monthly periods  : {len(TIME_PERIODS)} (Jan 2013 – Aug 2025)")
    print(f"  Potential exports: {total_potential:,}")
    print(f"    ├─ Monthly composites : {monthly_potential}")
    print(f"    ├─ Annual mosaics     : {annual_potential}")
    print(f"    └─ Seasonal (dry+wet) : {seasonal_potential}")
    print("=" * 72)

    init_ee()
    create_dirs()

    all_tasks     = []
    export_count  = 0
    skipped_count = 0

    # ════════════════════════════════════════════════════════════════════════
    # PART 1 — Monthly composites  (core of the 1 000+ count)
    # ════════════════════════════════════════════════════════════════════════
    print("\n[ PART 1 ]  Monthly composites — all years 2013-2025")
    print("-" * 72)

    for region_name, coords in REGIONS.items():
        roi = ee.Geometry.Rectangle(coords)
        print(f"\n  🌳 Region: {region_name}")

        for start_date, end_date, phase in tqdm(TIME_PERIODS,
                                                desc=f"  {region_name}",
                                                unit='month'):
            try:
                col   = get_s2_collection(roi, start_date, end_date)
                count = col.size().getInfo()

                if count < MIN_IMAGES_PER_PERIOD:
                    skipped_count += 1
                    continue

                composite = col.median().clip(roi)

                # GEE description: max 100 chars, no special characters
                year_month = start_date[:7].replace('-', '_')
                desc = f"MON_{region_name}_{year_month}_{phase[:8]}"[:90]

                ok = dispatch_export(composite, desc, roi,
                                     region_name, start_date, end_date,
                                     count, phase, all_tasks,
                                     thumb_subdir='exports')
                if ok:
                    export_count += 1
                else:
                    skipped_count += 1

                time.sleep(0.4)   # gentle rate-limiting

            except Exception as e:
                print(f"\n    ✗ Error [{start_date}]: {str(e)[:70]}")
                continue

    # ════════════════════════════════════════════════════════════════════════
    # PART 2 — Annual mosaics  (2013–2025, all 7 regions)
    # ════════════════════════════════════════════════════════════════════════
    print("\n\n[ PART 2 ]  Annual mosaics (2013–2025)")
    print("-" * 72)

    for region_name, coords in REGIONS.items():
        roi = ee.Geometry.Rectangle(coords)
        print(f"\n  🌳 Region: {region_name}")

        for year in range(2013, 2026):
            try:
                mosaic, count = create_annual_mosaic(roi, year)
                if count == 0:
                    print(f"    {year}: no scenes — skipped")
                    skipped_count += 1
                    continue

                desc = f"ANN_{region_name}_{year}"[:90]
                ok   = dispatch_export(mosaic, desc, roi,
                                       region_name,
                                       f'{year}-01-01', f'{year}-12-31',
                                       count, 'annual_mosaic', all_tasks,
                                       thumb_subdir='annual')
                if ok:
                    export_count += 1
                    print(f"    ✓ {year}: {count} scenes → {desc}")
                else:
                    skipped_count += 1

                time.sleep(0.4)

            except Exception as e:
                print(f"    ✗ {year}: {str(e)[:70]}")

    # ════════════════════════════════════════════════════════════════════════
    # PART 3 — Seasonal composites  (dry + wet, 2013–2024, all 7 regions)
    # ════════════════════════════════════════════════════════════════════════
    print("\n\n[ PART 3 ]  Seasonal composites (dry Nov–Apr + wet May–Oct)")
    print("-" * 72)

    for region_name, coords in REGIONS.items():
        roi = ee.Geometry.Rectangle(coords)
        print(f"\n  🌳 Region: {region_name}")

        for year in range(2013, 2025):
            for season in ('dry', 'wet'):
                try:
                    composite, count = create_seasonal_composite(roi, year, season)
                    if count == 0:
                        skipped_count += 1
                        continue

                    if season == 'dry':
                        s_date = f'{year}-11-01'
                        e_date = f'{year + 1}-04-30'
                    else:
                        s_date = f'{year}-05-01'
                        e_date = f'{year}-10-31'

                    desc  = f"SEAS_{season.upper()}_{region_name}_{year}"[:90]
                    ok    = dispatch_export(composite, desc, roi,
                                           region_name, s_date, e_date,
                                           count,
                                           f'{season}_season_composite',
                                           all_tasks,
                                           thumb_subdir='seasonal')
                    if ok:
                        export_count += 1
                        print(f"    ✓ {season.capitalize()} {year}: {count} scenes")
                    else:
                        skipped_count += 1

                    time.sleep(0.4)

                except Exception as e:
                    print(f"    ✗ {season} {year}: {str(e)[:70]}")

    # ════════════════════════════════════════════════════════════════════════
    # SUMMARY
    # ════════════════════════════════════════════════════════════════════════
    print("\n" + "=" * 72)
    print("  EXPORT SUMMARY")
    print("=" * 72)
    print(f"  ✓ Exports queued / downloaded : {export_count}")
    print(f"  ⊙ Skipped (no data / cached)  : {skipped_count}")
    print(f"  📊 Potential total             : {total_potential:,}")
    print()

    if EXPORT_MODE == 'drive':
        print("  📁 Destination : Google Drive  →  'Hasdeo_Training_Dataset/' folder")
        print("  ⏱  Large exports can take 10 min – several hours")
        print("  🔗 Monitor  : https://code.earthengine.google.com/tasks")
        save_task_ids(all_tasks)

    elif EXPORT_MODE == 'gcs':
        print(f"  📁 Destination : gs://{GCS_BUCKET}/hasdeo/")
        save_task_ids(all_tasks)

    else:
        print(f"  📁 PNG thumbnails saved to : {OUTPUT_DIR}/")

    save_metadata_csv()

    print()
    print("  DATASET BREAKDOWN")
    print(f"  ├─ Monthly composites   : up to {monthly_potential}  images")
    print(f"  ├─ Annual mosaics       : up to {annual_potential}  images")
    print(f"  ├─ Seasonal composites  : up to {seasonal_potential}  images")
    print(f"  ├─ Bands per image      : B2,B3,B4,B8,B8A,B11,B12,SCL + NDVI,NDWI,NBR,EVI")
    print(f"  └─ Format               : Cloud-Optimized GeoTIFF, EPSG:32644, {SCALE_METERS}m")
    print("=" * 72)
    print(f"\n  ✅ Script complete.  Check the task monitor for export progress.")


# ─────────────────────────────────────────────────────────────────────────────

if __name__ == '__main__':
    main()

  HASDEO FOREST — GeoTIFF DOWNLOADER v2  (TARGET: 1 000+ IMAGES)
  Export mode      : DRIVE
  Resolution       : 10 m
  Max cloud cover  : 20%
  Regions          : 7  → ['Hasdeo_Full', 'PEKB_Core', 'Kente_Extension', 'Control_Forest', 'Hasdeo_North', 'Hasdeo_South', 'Mining_Buffer']
  Monthly periods  : 152 (Jan 2013 – Aug 2025)
  Potential exports: 1,323
    ├─ Monthly composites : 1064
    ├─ Annual mosaics     : 91
    └─ Seasonal (dry+wet) : 168
✓ Earth Engine initialized | project: glacier-probe-model-475519
✓ Output directories ready under: hasdeo_training_dataset/

[ PART 1 ]  Monthly composites — all years 2013-2025
------------------------------------------------------------------------

  🌳 Region: Hasdeo_Full


  Hasdeo_Full: 100%|███████████████████████| 152/152 [03:51<00:00,  1.53s/month]



  🌳 Region: PEKB_Core


  PEKB_Core: 100%|█████████████████████████| 152/152 [03:06<00:00,  1.23s/month]



  🌳 Region: Kente_Extension


...


  🌳 Region: Hasdeo_South


  Hasdeo_South: 100%|██████████████████████| 152/152 [03:04<00:00,  1.21s/month]



  🌳 Region: Mining_Buffer


  Mining_Buffer: 100%|█████████████████████| 152/152 [03:03<00:00,  1.21s/month]




[ PART 2 ]  Annual mosaics (2013–2025)
------------------------------------------------------------------------

  🌳 Region: Hasdeo_Full
    2013: no scenes — skipped
    2014: no scenes — skipped
    2015: no scenes — skipped
    ✓ 2016: 3 scenes → ANN_Hasdeo_Full_2016
    ✓ 2017: 18 scenes → ANN_Hasdeo_Full_2017
    ✓ 2018: 24 scenes → ANN_Hasdeo_Full_2018
    ✓ 2019: 71 scenes → ANN_Hasdeo_Full_2019
    ✓ 2020: 63 scenes → ANN_Hasdeo_Full_2020
    ✓ 2021: 74 scenes → ANN_Hasdeo_Full_2021
    ✓ 2022: 73 scenes → ANN_Hasdeo_Full_2022
    ✓ 2023: 65 scenes → ANN_Hasdeo_Full_2023
    ✓ 2024: 64 scenes → ANN_Hasdeo_Full_2024
    ✓ 2025: 89 scenes → ANN_Hasdeo_Full_2025

  🌳 Region: PEKB_Core
    2013: no scenes — skipped
    2014: no scenes — skipped
    2015: no scenes — skipped
    ✓ 2016: 2 scenes → ANN_PEKB_Core_2016
    ✓ 2017: 9 scenes → ANN_PEKB_Core_2017
    ✓ 2018: 15 scenes → ANN_PEKB_Core_2018
    ✓ 2019: 35 scenes → ANN_PEKB_Core_2019
    ✓ 2020: 31 scenes → ANN_PEKB_Core_2

In [16]:
"""
Kangaroo Island Black Summer Bushfire Image Downloader
Downloads RGB satellite imagery for Kangaroo Island, South Australia
Covering the 2019-20 Black Summer bushfire period and recovery
Images are saved as PNG with proper color scaling for visualization.
"""

import ee
import os
import time
from pathlib import Path
import requests

# --- CONFIGURATION ---

PROJECT_ID = 'glacier-probe-model-475519'
OUTPUT_DIR = 'kangaroo_island_black_summer'
NUM_IMAGES = 100  # More images to capture pre-fire, during, and post-fire periods
SCALE = 30  # 30m resolution for manageable file sizes
MAX_CLOUD_COVER = 100  # Allow ALL images including heavy smoke/clouds

# Kangaroo Island Area of Interest - Black Summer Fire Zone
# Coordinates converted from DMS to decimal degrees:
# Latitude: 35°33'41"S to 36°05'12"S = -35.5614 to -36.0867
# Longitude: 136°32'04"E to 138°00'00"E = 136.5344 to 138.0000
KANGAROO_ISLAND_REGION = {
    'Kangaroo_Island_Fire_Zone': [136.5344, -36.0867, 138.0000, -35.5614]
}

# Black Summer bushfires on Kangaroo Island:
# - Started: December 20, 2019 (lightning strikes)
# - Major fires: December 30, 2019 - February 6, 2020
# - Declared safe: February 6, 2020
# Year ranges: 5 years before (2014-2019) and 5 years after (2020-2025)
YEAR_RANGES = [
    # Pre-fire period (5 years before)
    ('2014-01-01', '2014-12-31'),
    ('2015-01-01', '2015-12-31'),
    ('2016-01-01', '2016-12-31'),
    ('2017-01-01', '2017-12-31'),
    ('2018-01-01', '2018-12-31'),
    # Critical fire year
    ('2019-01-01', '2019-12-19'),  # Pre-fire 2019
    ('2019-12-20', '2020-02-06'),  # During fire (Dec 20, 2019 - Feb 6, 2020)
    ('2020-02-07', '2020-12-31'),  # Post-fire recovery 2020
    # Post-fire recovery period (4 more years)
    ('2021-01-01', '2021-12-31'),
    ('2022-01-01', '2022-12-31'),
    ('2023-01-01', '2023-12-31'),
    ('2024-01-01', '2024-12-31'),
]

# --- INITIALIZATION ---

try:
    ee.Initialize(project=PROJECT_ID)
    print(f"✓ Earth Engine initialized for project: {PROJECT_ID}")
except Exception as e:
    print(f"⚠ Initialization failed. Attempting authentication...")
    ee.Authenticate()
    ee.Initialize(project=PROJECT_ID)
    print(f"✓ Authentication successful. Earth Engine initialized.")

# --- UTILITY FUNCTIONS ---

def create_output_dirs():
    """Create output directory structure"""
    Path(OUTPUT_DIR).mkdir(exist_ok=True)
    Path(f"{OUTPUT_DIR}/rgb_images").mkdir(exist_ok=True)
    Path(f"{OUTPUT_DIR}/metadata").mkdir(exist_ok=True)
    print(f"✓ Created output directories in {OUTPUT_DIR}/")

def get_satellite_collection(start_date, end_date, roi, max_cloud_cover):
    """Get Sentinel-2 collection WITHOUT cloud masking to preserve smoke/fire effects"""
    # NO CLOUD MASKING - We want to see smoke, clouds, and fire effects!
    sentinel2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
        .filterBounds(roi) \
        .filterDate(start_date, end_date) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', max_cloud_cover))

    s2_count = sentinel2.size().getInfo()
    
    if s2_count > 0:
        return sentinel2, 'sentinel2'
    else:
        return None, None

def download_rgb_image(image, region, filename):
    """Downloads RGB image in viewable PNG format"""
    try:
        # Select RGB bands (B4=Red, B3=Green, B2=Blue)
        rgb = image.select(['B4', 'B3', 'B2'])
        
        # Normalize to 0-255 range for viewing
        # Sentinel-2 values are 0-10000, we'll use 0-3000 as typical range
        rgb_vis = rgb.visualize(
            min=0,
            max=3000,
            bands=['B4', 'B3', 'B2']
        )
        
        # Get thumbnail URL (this returns a viewable image)
        url = rgb_vis.getThumbURL({
            'region': region,
            'dimensions': 2048,  # Higher resolution for large area (2048x2048)
            'format': 'png'
        })
        
        # Download and save as PNG
        response = requests.get(url, timeout=300)
        if response.status_code == 200:
            # Save directly as PNG
            with open(filename, 'wb') as f:
                f.write(response.content)
            return True
        else:
            print(f"✗ HTTP Error {response.status_code}")
            return False
            
    except Exception as e:
        error_msg = str(e)
        print(f"✗ Error: {error_msg[:100]}...")
        return False

def save_metadata(image, filename, region_name, period_label):
    """Save image metadata"""
    try:
        props = image.getInfo()['properties']
        metadata_file = filename
        
        with open(metadata_file, 'w') as f:
            f.write(f"Region: {region_name}\n")
            f.write(f"Location: Kangaroo Island, South Australia\n")
            f.write(f"Event: Black Summer Bushfires (2019-20)\n")
            f.write(f"Period: {period_label}\n")
            f.write(f"Satellite: SENTINEL-2\n")
            f.write(f"Image Type: RGB (True Color)\n")
            f.write(f"Format: PNG (Viewable)\n")
            
            date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd').getInfo()
            f.write(f"Date: {date}\n")
            f.write(f"Cloud Cover: {props.get('CLOUDY_PIXEL_PERCENTAGE', 'N/A')}%\n")
            f.write(f"Product ID: {props.get('PRODUCT_ID', 'N/A')}\n")
            f.write(f"Bands: R=B4 (Red), G=B3 (Green), B=B2 (Blue)\n")
            f.write(f"Resolution: {SCALE}m\n")
            f.write(f"Processing: NO cloud masking - smoke and fire effects preserved\n")
            f.write(f"\nContext: Area burned ~211,500 ha (48% of island)\n")
            f.write(f"Fire Period: Dec 20, 2019 - Feb 6, 2020\n")
            
    except Exception as e:
        print(f"  ⚠ Metadata warning: {str(e)[:30]}")

def get_period_label(start_date, end_date):
    """Get descriptive label for the time period"""
    if '2019-12-20' in start_date and '2020-02-06' in end_date:
        return "DURING FIRE (Dec 20, 2019 - Feb 6, 2020)"
    elif int(start_date[:4]) < 2019 or (start_date[:4] == '2019' and '12-19' in start_date):
        return f"PRE-FIRE ({start_date[:4]})"
    elif int(start_date[:4]) >= 2020:
        years_after = int(start_date[:4]) - 2020
        return f"POST-FIRE RECOVERY (+{years_after} years - {start_date[:4]})"
    return start_date[:4]

# --- MAIN EXECUTION ---

def main_kangaroo_island():
    """Main download function for Kangaroo Island Black Summer"""
    print("=" * 80)
    print("KANGAROO ISLAND BLACK SUMMER BUSHFIRE IMAGE DOWNLOADER")
    print(f"Coordinates: {KANGAROO_ISLAND_REGION['Kangaroo_Island_Fire_Zone']}")
    print("Fire Period: December 20, 2019 - February 6, 2020")
    print("Coverage: 5 years pre-fire (2014-2019) + fire period + 5 years post-fire (2020-2024)")
    print("=" * 80)
    
    create_output_dirs()
    
    downloaded_count = 0
    failed_count = 0
    
    print(f"\nTarget: {NUM_IMAGES} images (Max Cloud Cover: {MAX_CLOUD_COVER}%)")
    print(f"Time periods: {len(YEAR_RANGES)}")
    print(f"Resolution: {SCALE}m | Format: PNG (Viewable RGB)\n")
    
    region_name = 'Kangaroo_Island_Fire_Zone'
    coords = KANGAROO_ISLAND_REGION[region_name]
    roi = ee.Geometry.Rectangle(coords)

    print(f" Starting Downloads for: Kangaroo Island Black Summer")
    print("-" * 80)
    
    # Iterate through each year range
    for start_date, end_date in YEAR_RANGES:
        if downloaded_count >= NUM_IMAGES:
            break
        
        period_label = get_period_label(start_date, end_date)
        
        # Get the satellite collection for the period
        collection, satellite_type = get_satellite_collection(start_date, end_date, roi, MAX_CLOUD_COVER)
        
        if collection is None:
            print(f"  {period_label}: 0 images found")
            continue
        
        try:
            count = collection.size().getInfo()
            if count == 0:
                print(f"  {period_label}: 0 images found")
                continue
                
            print(f"  {period_label}: {count} images available")
            
            # Adjust number of images per period
            # More images during fire period, fewer for other years
            if 'DURING FIRE' in period_label:
                images_per_period = min(20, count, NUM_IMAGES - downloaded_count)
            else:
                images_per_period = min(8, count, NUM_IMAGES - downloaded_count)
                
            if images_per_period <= 0:
                break
                
            images = collection.limit(images_per_period).toList(images_per_period)
            
            for i in range(images_per_period):
                if downloaded_count >= NUM_IMAGES:
                    break
                    
                image = ee.Image(images.get(i))
                
                # Get date
                date_acquired = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd').getInfo()
                
                # Filename construction (PNG format)
                period_prefix = period_label.split('(')[0].strip().replace(' ', '_').replace('-', '')
                filename = f"{OUTPUT_DIR}/rgb_images/KI_{date_acquired}_{period_prefix}_RGB.png"
                metadata_filename = f"{OUTPUT_DIR}/metadata/KI_{date_acquired}_{period_prefix}_metadata.txt"
                
                # Skip if exists
                if os.path.exists(filename):
                    downloaded_count += 1
                    print(f"    ⊙ {date_acquired} (exists)")
                    continue
                
                print(f"    ↓ {date_acquired}...", end=' ')
                
                if download_rgb_image(image, roi, filename):
                    save_metadata(image, metadata_filename, region_name, period_label)
                    downloaded_count += 1
                    print(f"✓ ({downloaded_count}/{NUM_IMAGES})")
                else:
                    failed_count += 1
                
                time.sleep(2)
                
        except Exception as e:
            print(f"  ✗ Error in collection processing: {str(e)[:50]}...")
            continue
            
    # Summary
    print("\n" + "=" * 80)
    print("DOWNLOAD COMPLETE - KANGAROO ISLAND BLACK SUMMER DATASET")
    print("=" * 80)
    print(f"✓ Downloaded: {downloaded_count} images")
    print(f"✗ Failed: {failed_count}")
    print(f" Location: {OUTPUT_DIR}/rgb_images/")
    print(f"\n Dataset Coverage:")
    print(f"   • Pre-fire baseline: 2014-2019 (5 years)")
    print(f"   • Active fire period: Dec 20, 2019 - Feb 6, 2020")
    print(f"   • Post-fire recovery: 2020-2024 (5 years)")
    print(f"   • Area burned: ~211,500 hectares (48% of island)")
    print(f"\n✓ Images are in standard PNG format and viewable!")
    print("You can open them with any image viewer or photo app.")
    print("=" * 80)

if __name__ == "__main__":
    main_kangaroo_island()

✓ Earth Engine initialized for project: glacier-probe-model-475519
KANGAROO ISLAND BLACK SUMMER BUSHFIRE IMAGE DOWNLOADER
Coordinates: [136.5344, -36.0867, 138.0, -35.5614]
Fire Period: December 20, 2019 - February 6, 2020
Coverage: 5 years pre-fire (2014-2019) + fire period + 5 years post-fire (2020-2024)
✓ Created output directories in kangaroo_island_black_summer/

Target: 100 images (Max Cloud Cover: 100%)
Time periods: 12
Resolution: 30m | Format: PNG (Viewable RGB)

🔥 Starting Downloads for: Kangaroo Island Black Summer
--------------------------------------------------------------------------------
  PRE-FIRE (2014): 0 images found
  PRE-FIRE (2015): 0 images found
  PRE-FIRE (2016): 0 images found
  PRE-FIRE (2017): 4 images available
    ↓ 2017-03-08... ✓ (1/100)
    ↓ 2017-07-21... ✓ (2/100)
    ⊙ 2017-07-21 (exists)
    ↓ 2017-12-26... ✓ (4/100)
  PRE-FIRE (2018): 28 images available
    ↓ 2018-02-06... ✓ (5/100)
    ⊙ 2018-02-06 (exists)
    ↓ 2018-02-26... ✓ (7/100)
    ⊙ 

In [18]:
"""
Greater Sydney Blue Mountains Fringe Satellite Image Downloader
Downloads RGB satellite imagery for the Blue Mountains/Warragamba Dam Catchment Area
Captures: Bushfire (Black Summer 2019-20), Drought, Regrowth, and Deforestation
Images are saved as PNG with proper color scaling for visualization.
"""

import ee
import os
import time
from pathlib import Path
import requests

# --- CONFIGURATION (MODIFIED for Sydney Blue Mountains Fringe) ---

PROJECT_ID = 'glacier-probe-model-475519'
OUTPUT_DIR = 'sydney_blue_mountains_fringe_rgb'
NUM_IMAGES = 75  # 15 images per sub-region
SCALE = 30  # Increased to 30m to ensure successful downloads
MAX_CLOUD_COVER = 100  # Allow all images including smoke/clouds during fires

# Greater Sydney Blue Mountains Fringe - Multiple Sub-Regions
# Multiple smaller regions to ensure visible imagery capture
# Coordinates: [lon_min, lat_min, lon_max, lat_max]
SYDNEY_REGIONS = {
    'Blue_Mts_Katoomba': [150.25, -33.75, 150.35, -33.65],  # Katoomba/Three Sisters area
    'Blue_Mts_Wentworth_Falls': [150.35, -33.75, 150.45, -33.65],  # Wentworth Falls
    'Warragamba_Dam_North': [150.55, -33.95, 150.65, -33.85],  # North of dam
    'Warragamba_Dam_South': [150.55, -34.05, 150.65, -33.95],  # South of dam
    'Penrith_Urban_Edge': [150.65, -33.80, 150.75, -33.70],  # Urban fringe
}

# Date ranges specifically targeting the Black Summer event and recovery/drought periods
YEAR_RANGES = [
    # Pre-fire/drought baseline
    ('2018-01-01', '2019-01-01'), 
    # Peak drought and fire period (Black Summer)
    ('2019-06-01', '2020-06-01'), 
    # Immediate post-fire and start of regrowth
    ('2020-06-01', '2021-06-01'), 
    # Continued recovery and expansion monitoring
    ('2021-06-01', '2022-06-01'),
    ('2022-06-01', '2023-06-01'),
    ('2023-06-01', '2024-06-01'),
    ('2024-06-01', '2025-06-01'),
]

# --- INITIALIZATION ---

try:
    ee.Initialize(project=PROJECT_ID)
    print(f"✓ Earth Engine initialized for project: {PROJECT_ID}")
except Exception as e:
    print(f"⚠ Initialization failed. Attempting authentication...")
    ee.Authenticate()
    ee.Initialize(project=PROJECT_ID)
    print(f"✓ Authentication successful. Earth Engine initialized.")

# --- UTILITY FUNCTIONS ---

def create_output_dirs():
    """Create output directory structure"""
    Path(OUTPUT_DIR).mkdir(exist_ok=True)
    Path(f"{OUTPUT_DIR}/rgb_images").mkdir(exist_ok=True)
    Path(f"{OUTPUT_DIR}/metadata").mkdir(exist_ok=True)
    print(f"✓ Created output directories in {OUTPUT_DIR}/")

def get_satellite_collection(start_date, end_date, roi, max_cloud_cover):
    """Get Sentinel-2 collection WITHOUT cloud masking to preserve all effects"""
    sentinel2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
        .filterBounds(roi) \
        .filterDate(start_date, end_date) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', max_cloud_cover))

    s2_count = sentinel2.size().getInfo()
    
    if s2_count > 0:
        return sentinel2, 'sentinel2'
    else:
        return None, None

def download_rgb_image(image, region, filename):
    """Downloads RGB image in viewable PNG format"""
    try:
        # Select RGB bands (B4=Red, B3=Green, B2=Blue)
        rgb = image.select(['B4', 'B3', 'B2'])
        
        # Normalize to 0-255 range for viewing
        rgb_vis = rgb.visualize(
            min=0,
            max=3000,
            bands=['B4', 'B3', 'B2']
        )
        
        # Get thumbnail URL with adjusted dimensions
        url = rgb_vis.getThumbURL({
            'region': region,
            'dimensions': 512,  # Reduced for better compatibility
            'format': 'png'
        })
        
        # Download and save as PNG
        response = requests.get(url, timeout=300)
        if response.status_code == 200:
            with open(filename, 'wb') as f:
                f.write(response.content)
            return True
        else:
            print(f"✗ HTTP Error {response.status_code}")
            return False
            
    except Exception as e:
        error_msg = str(e)
        print(f"✗ Error: {error_msg[:100]}...")
        return False

def save_metadata(image, filename, region_name, period_label):
    """Save image metadata"""
    try:
        props = image.getInfo()['properties']
        metadata_file = filename
        
        with open(metadata_file, 'w') as f:
            f.write(f"Region: {region_name}\n")
            f.write(f"Location: Greater Sydney - Blue Mountains Fringe & Warragamba Catchment\n")
            f.write(f"Research Focus: Bushfire, Drought, Regrowth, Deforestation\n")
            f.write(f"Period: {period_label}\n\n")
            
            f.write(f"Satellite: SENTINEL-2\n")
            f.write(f"Image Type: RGB (True Color)\n")
            f.write(f"Format: PNG (Viewable)\n")
            
            date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd').getInfo()
            f.write(f"Date: {date}\n")
            f.write(f"Cloud Cover: {props.get('CLOUDY_PIXEL_PERCENTAGE', 'N/A')}%\n")
            f.write(f"Product ID: {props.get('PRODUCT_ID', 'N/A')}\n")
            f.write(f"Bands: R=B4 (Red), G=B3 (Green), B=B2 (Blue)\n")
            f.write(f"Resolution: {SCALE}m\n")
            f.write(f"Processing: NO cloud masking - smoke and fire effects preserved\n")
            
            f.write(f"\n=== RESEARCH CONTEXT ===\n")
            f.write(f"Black Summer Fires: 2019-2020\n")
            f.write(f"Key Indices for Analysis:\n")
            f.write(f"  • NBR (Normalized Burn Ratio): Fire severity\n")
            f.write(f"  • NDVI (Normalized Difference Vegetation Index): Vegetation health/regrowth\n")
            f.write(f"  • EVI (Enhanced Vegetation Index): Drought impact\n")
            f.write(f"  • NDMI (Normalized Difference Moisture Index): Water stress\n")
            f.write(f"  • NDWI (Normalized Difference Water Index): Drought monitoring\n")
            
    except Exception as e:
        print(f"  ⚠ Metadata warning: {str(e)[:30]}")

def get_period_label(start_date, end_date):
    """Get descriptive label for the time period"""
    if '2018' in start_date:
        return "PRE-FIRE BASELINE (2018)"
    elif '2019-06' in start_date and '2020-06' in end_date:
        return "DROUGHT & BLACK SUMMER FIRES (2019-2020)"
    elif '2020-06' in start_date and '2021' in end_date:
        return "IMMEDIATE POST-FIRE RECOVERY (2020-2021)"
    elif '2021' in start_date:
        return f"POST-FIRE RECOVERY & MONITORING ({start_date[:4]}-{end_date[:4]})"
    elif '2022' in start_date:
        return f"REGROWTH & DEFORESTATION MONITORING ({start_date[:4]}-{end_date[:4]})"
    elif '2023' in start_date or '2024' in start_date:
        return f"LONG-TERM RECOVERY ({start_date[:4]}-{end_date[:4]})"
    return f"{start_date[:4]}-{end_date[:4]}"

# --- MAIN EXECUTION ---

def main_sydney():
    """Main download function for Sydney Blue Mountains Fringe - Multiple Regions"""
    print("=" * 80)
    print("GREATER SYDNEY BLUE MOUNTAINS FRINGE IMAGE DOWNLOADER")
    print("Multiple Sub-Regions for Complete Coverage")
    print("=" * 80)
    print(f"📍 Regions: {len(SYDNEY_REGIONS)}")
    for name, coords in SYDNEY_REGIONS.items():
        print(f"   • {name}: {coords}")
    print(f" Event: Black Summer Bushfires (2019-2020)")
    print(f" Context: Prolonged drought + catastrophic fire + urban expansion")
    print(f" Research: Bushfire, Drought, Regrowth, Deforestation")
    print("=" * 80)
    
    create_output_dirs()
    
    total_downloaded = 0
    total_failed = 0
    
    print(f"\nTarget: {NUM_IMAGES} images total (~{NUM_IMAGES // len(SYDNEY_REGIONS)} per region)")
    print(f"Time periods: {len(YEAR_RANGES)}")
    print(f"Resolution: {SCALE}m | Format: PNG (Viewable RGB)\n")
    
    images_per_region = NUM_IMAGES // len(SYDNEY_REGIONS)
    
    # Process each sub-region
    for region_name, coords in SYDNEY_REGIONS.items():
        print(f"\n{'='*80}")
        print(f"🌳 Processing Region: {region_name}")
        print(f"📍 Coordinates: {coords}")
        print(f"🎯 Target: {images_per_region} images")
        print("-" * 80)
        
        roi = ee.Geometry.Rectangle(coords)
        downloaded_count = 0
        failed_count = 0
        
        # Iterate through each year range for this region
        for start_date, end_date in YEAR_RANGES:
            if downloaded_count >= images_per_region:
                break
            
            period_label = get_period_label(start_date, end_date)
            
            # Get the satellite collection for the period
            collection, satellite_type = get_satellite_collection(start_date, end_date, roi, MAX_CLOUD_COVER)
            
            if collection is None:
                print(f"  {period_label}: 0 images found")
                continue
            
            try:
                count = collection.size().getInfo()
                if count == 0:
                    print(f"  {period_label}: 0 images found")
                    continue
                    
                print(f"  {period_label}: {count} images available")
                
                # Images per period
                if 'BLACK SUMMER' in period_label:
                    images_per_period = min(5, count, images_per_region - downloaded_count)
                else:
                    images_per_period = min(3, count, images_per_region - downloaded_count)
                    
                if images_per_period <= 0:
                    break
                    
                images = collection.limit(images_per_period).toList(images_per_period)
                
                for i in range(images_per_period):
                    if downloaded_count >= images_per_region:
                        break
                        
                    image = ee.Image(images.get(i))
                    
                    # Get date
                    date_acquired = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd').getInfo()
                    
                    # Filename construction
                    period_code = period_label.split('(')[0].strip().replace(' ', '_').replace('&', '').replace('-', '')
                    filename = f"{OUTPUT_DIR}/rgb_images/{region_name}_{date_acquired}_{period_code}_RGB.png"
                    metadata_filename = f"{OUTPUT_DIR}/metadata/{region_name}_{date_acquired}_{period_code}_meta.txt"
                    
                    # Skip if exists
                    if os.path.exists(filename):
                        downloaded_count += 1
                        print(f"    ⊙ {date_acquired} (exists)")
                        continue
                    
                    print(f"    ↓ {date_acquired}...", end=' ')
                    
                    if download_rgb_image(image, roi, filename):
                        save_metadata(image, metadata_filename, region_name, period_label)
                        downloaded_count += 1
                        print(f"✓ ({downloaded_count}/{images_per_region})")
                    else:
                        failed_count += 1
                    
                    time.sleep(1.5)
                    
            except Exception as e:
                print(f"  ✗ Error in collection processing: {str(e)[:50]}...")
                continue
        
        total_downloaded += downloaded_count
        total_failed += failed_count
        print(f"\n✓ {region_name}: {downloaded_count} downloaded, {failed_count} failed")
    
    # Final Summary
    print("\n" + "=" * 80)
    print("DOWNLOAD COMPLETE - SYDNEY BLUE MOUNTAINS MULTI-REGION DATASET")
    print("=" * 80)
    print(f"✓ Total Downloaded: {total_downloaded}/{NUM_IMAGES} images")
    print(f"✗ Total Failed: {total_failed}")
    print(f" Location: {OUTPUT_DIR}/rgb_images/")
    print(f"\n Dataset Coverage:")
    print(f"   • Pre-fire baseline: 2018")
    print(f"   • Drought & Black Summer fires: 2019-2020")
    print(f"   • Post-fire recovery: 2020-2021")
    print(f"   • Long-term monitoring: 2021-2025")
    print(f"\n Sub-Regions Captured:")
    for name in SYDNEY_REGIONS.keys():
        print(f"   • {name}")
    print(f"\n Recommended Analysis Indices:")
    print(f"   • NBR (Normalized Burn Ratio): B8-B12 / B8+B12")
    print(f"     → Fire severity mapping, burn scar detection")
    print(f"   • NDVI (Vegetation Index): B8-B4 / B8+B4")
    print(f"     → Vegetation health, regrowth monitoring")
    print(f"   • EVI (Enhanced Vegetation): 2.5*(B8-B4) / (B8+6*B4-7.5*B2+1)")
    print(f"     → Drought stress, canopy density")
    print(f"   • NDMI (Moisture Index): B8-B11 / B8+B11")
    print(f"     → Water stress, drought impact")
    print(f"   • NDWI (Water Index): B3-B8 / B3+B8")
    print(f"     → Drought severity, water availability")
    print(f"\n✓ Images are in standard PNG format and viewable!")
    print("All smoke, fire, and atmospheric effects are preserved.")
    print("=" * 80)

if __name__ == "__main__":
    main_sydney()

✓ Earth Engine initialized for project: glacier-probe-model-475519
🇦🇺 GREATER SYDNEY BLUE MOUNTAINS FRINGE IMAGE DOWNLOADER
Multiple Sub-Regions for Complete Coverage
📍 Regions: 5
   • Blue_Mts_Katoomba: [150.25, -33.75, 150.35, -33.65]
   • Blue_Mts_Wentworth_Falls: [150.35, -33.75, 150.45, -33.65]
   • Warragamba_Dam_North: [150.55, -33.95, 150.65, -33.85]
   • Warragamba_Dam_South: [150.55, -34.05, 150.65, -33.95]
   • Penrith_Urban_Edge: [150.65, -33.8, 150.75, -33.7]
🔥 Event: Black Summer Bushfires (2019-2020)
🌵 Context: Prolonged drought + catastrophic fire + urban expansion
📊 Research: Bushfire, Drought, Regrowth, Deforestation
✓ Created output directories in sydney_blue_mountains_fringe_rgb/

Target: 75 images total (~15 per region)
Time periods: 7
Resolution: 30m | Format: PNG (Viewable RGB)


🌳 Processing Region: Blue_Mts_Katoomba
📍 Coordinates: [150.25, -33.75, 150.35, -33.65]
🎯 Target: 15 images
-------------------------------------------------------------------------------

In [5]:
"""
Hasdeo Forest GeoTIFF Downloader - FINAL FIXED VERSION
=======================================================
FIX: Persistent "Float64 and Float32 incompatible types" error.

ROOT CAUSE (deeper than previous fix):
  - SCL band             → UInt8  (integer)
  - Raw S2 bands         → UInt16 / Float32
  - normalizedDifference → Float64
  - .toFloat() on the whole composite does NOT always propagate
    correctly when bands were added at different pipeline stages.

SOLUTION: Cast each band individually AND cast entire composite
  at export time using .select().toFloat() on every band explicitly,
  then drop SCL before export OR cast SCL separately to Float32.
"""

import ee
import os
import csv
import time
import json
import requests
from pathlib import Path
from datetime import datetime
from tqdm import tqdm

# ─────────────────────────────────────────────────────────────────────────────
# CONFIGURATION
# ─────────────────────────────────────────────────────────────────────────────

PROJECT_ID            = 'glacier-probe-model-475519'
GCS_BUCKET            = ''
OUTPUT_DIR            = 'hasdeo_training_dataset'
SCALE_METERS          = 10
MAX_CLOUD_COVER       = 20
MIN_IMAGES_PER_PERIOD = 1
EXPORT_MODE           = 'drive'   # 'drive' | 'gcs' | 'local_thumb'

REGIONS = {
    'Hasdeo_Full':     [82.60, 22.80, 83.00, 23.20],
    'PEKB_Core':       [82.70, 22.90, 82.85, 23.05],
    'Kente_Extension': [82.65, 23.05, 82.80, 23.20],
    'Control_Forest':  [82.85, 23.00, 83.00, 23.15],
    'Hasdeo_North':    [82.60, 23.10, 82.80, 23.30],
    'Hasdeo_South':    [82.70, 22.70, 82.90, 22.90],
    'Mining_Buffer':   [82.75, 22.85, 82.95, 23.05],
}

# NOTE: SCL removed from S2_BANDS here — it is handled separately
# as Float32 to avoid UInt8 vs Float32 conflict
S2_BANDS = ['B2', 'B3', 'B4', 'B8', 'B8A', 'B11', 'B12']

# All final band names that will appear in every exported GeoTIFF
ALL_EXPORT_BANDS = ['B2', 'B3', 'B4', 'B8', 'B8A', 'B11', 'B12',
                    'SCL', 'NDVI', 'NDWI', 'NBR', 'EVI']

# ─────────────────────────────────────────────────────────────────────────────
# TIME PERIODS
# ─────────────────────────────────────────────────────────────────────────────

def build_monthly_periods():
    phase_map = {
        range(2013, 2016): 'init_clr',
        range(2016, 2020): 'rapid_exp',
        range(2020, 2023): 'slowdown',
        range(2023, 2026): 'renewed',
    }
    def get_phase(year):
        for yr_range, label in phase_map.items():
            if year in yr_range:
                return label
        return 'unknown'

    periods = []
    for year in range(2013, 2026):
        for month in range(1, 13):
            if year == 2025 and month > 8:
                break
            start = f'{year}-{month:02d}-01'
            end   = f'{year + 1}-01-01' if month == 12 else f'{year}-{month + 1:02d}-01'
            periods.append((start, end, get_phase(year)))
    return periods

TIME_PERIODS = build_monthly_periods()

# ─────────────────────────────────────────────────────────────────────────────
# INIT
# ─────────────────────────────────────────────────────────────────────────────

def init_ee():
    try:
        ee.Initialize(project=PROJECT_ID)
        print(f"✓ Earth Engine initialized | project: {PROJECT_ID}")
    except Exception:
        print("⚠  Re-authenticating...")
        ee.Authenticate()
        ee.Initialize(project=PROJECT_ID)
        print("✓ Authenticated and initialized.")

def create_dirs():
    for sub in ['exports', 'metadata', 'logs', 'annual', 'seasonal']:
        Path(f"{OUTPUT_DIR}/{sub}").mkdir(parents=True, exist_ok=True)
    print(f"✓ Directories ready: {OUTPUT_DIR}/")

# ─────────────────────────────────────────────────────────────────────────────
# CLOUD MASKING
# ─────────────────────────────────────────────────────────────────────────────

def mask_s2_clouds(image):
    qa  = image.select('QA60')
    qa_mask = (qa.bitwiseAnd(1 << 10).eq(0)
                 .And(qa.bitwiseAnd(1 << 11).eq(0)))
    scl = image.select('SCL')
    scl_mask = (scl.neq(3).And(scl.neq(8))
                          .And(scl.neq(9))
                          .And(scl.neq(10))
                          .And(scl.neq(11)))
    return image.updateMask(qa_mask.And(scl_mask))

# ─────────────────────────────────────────────────────────────────────────────
# INDEX COMPUTATION — each band individually cast to Float32
# ─────────────────────────────────────────────────────────────────────────────

def add_indices(image):
    """
    Each index is cast to Float32 with .toFloat() immediately after
    computation — before being added to the image stack.
    This prevents Float64 bleeding into the composite.
    """
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI').toFloat()
    ndwi = image.normalizedDifference(['B3', 'B8']).rename('NDWI').toFloat()
    nbr  = image.normalizedDifference(['B8', 'B12']).rename('NBR').toFloat()
    evi  = (image.expression(
                '2.5 * ((NIR - RED) / (NIR + 6.0 * RED - 7.5 * BLUE + 1.0))',
                {'NIR':  image.select('B8').toFloat(),
                 'RED':  image.select('B4').toFloat(),
                 'BLUE': image.select('B2').toFloat()}
            ).rename('EVI').toFloat())
    return image.addBands([ndvi, ndwi, nbr, evi])

# ─────────────────────────────────────────────────────────────────────────────
# ══════════════════════════════════════════════════════════════════════════════
# THE DEFINITIVE FIX  →  force_float32()
# ══════════════════════════════════════════════════════════════════════════════
# Strategy:
#   1. Select each band by name one at a time
#   2. Cast every single band to Float32 explicitly
#   3. Combine back into a single image with addBands
#   4. This guarantees ALL bands are Float32 — no exceptions
#
# Why previous .toFloat() on the composite failed:
#   GEE's .toFloat() casts the IMAGE object, but internally bands
#   added via .addBands() at different stages can retain their
#   original type metadata. Selecting + casting band-by-band is
#   the only guaranteed approach.
# ══════════════════════════════════════════════════════════════════════════════

def force_float32(image, band_names):
    """
    Cast every band in band_names to Float32 individually,
    then stack them back into a single image.
    Guaranteed to produce a type-homogeneous image.
    """
    bands = [image.select([b]).toFloat() for b in band_names]
    result = bands[0]
    for b in bands[1:]:
        result = result.addBands(b)
    return result

# ─────────────────────────────────────────────────────────────────────────────
# COLLECTION BUILDER
# ─────────────────────────────────────────────────────────────────────────────

def get_s2_collection(roi, start_date, end_date):
    """
    Returns cloud-masked, index-enriched Sentinel-2 collection.
    SCL is fetched alongside for masking, then kept as a band (cast to float).
    """
    return (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
              .filterBounds(roi)
              .filterDate(start_date, end_date)
              .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', MAX_CLOUD_COVER))
              .select(S2_BANDS + ['SCL', 'QA60'])
              .map(mask_s2_clouds)
              .map(add_indices))

# ─────────────────────────────────────────────────────────────────────────────
# COMPOSITE BUILDER — includes the Float32 guarantee
# ─────────────────────────────────────────────────────────────────────────────

def build_composite(col, roi):
    """
    1. Compute median composite
    2. Clip to ROI
    3. Force every band to Float32 (THE KEY STEP)
    """
    composite = col.median().clip(roi)
    return force_float32(composite, ALL_EXPORT_BANDS)

def create_seasonal_composite(roi, year, season='dry'):
    if season == 'dry':
        start, end = f'{year}-11-01', f'{year + 1}-04-30'
    else:
        start, end = f'{year}-05-01', f'{year}-10-31'
    col   = get_s2_collection(roi, start, end)
    count = col.size().getInfo()
    return build_composite(col, roi), count

def create_annual_mosaic(roi, year):
    col   = get_s2_collection(roi, f'{year}-01-01', f'{year}-12-31')
    count = col.size().getInfo()
    return build_composite(col, roi), count

# ─────────────────────────────────────────────────────────────────────────────
# EXPORT FUNCTIONS
# ─────────────────────────────────────────────────────────────────────────────

def export_to_drive(image, description, region):
    task = ee.batch.Export.image.toDrive(
        image          = image,           # already Float32 guaranteed
        description    = description,
        folder         = 'Hasdeo_Training_Dataset',
        fileNamePrefix = description,
        region         = region,
        scale          = SCALE_METERS,
        crs            = 'EPSG:32644',
        fileFormat     = 'GeoTIFF',
        maxPixels      = 1e10,
        formatOptions  = {'cloudOptimized': True}
    )
    task.start()
    return task

def export_to_gcs(image, description, region, bucket):
    task = ee.batch.Export.image.toCloudStorage(
        image          = image,
        description    = description,
        bucket         = bucket,
        fileNamePrefix = f'hasdeo/{description}',
        region         = region,
        scale          = SCALE_METERS,
        crs            = 'EPSG:32644',
        fileFormat     = 'GeoTIFF',
        maxPixels      = 1e10,
        formatOptions  = {'cloudOptimized': True}
    )
    task.start()
    return task

def download_local_thumb(image, region, filepath, size=512):
    try:
        vis = image.visualize(bands=['B4', 'B3', 'B2'], min=0, max=3000)
        url = vis.getThumbURL({'region': region, 'dimensions': size, 'format': 'png'})
        r   = requests.get(url, timeout=120)
        if r.status_code == 200:
            with open(filepath, 'wb') as f:
                f.write(r.content)
            return True
    except Exception as e:
        print(f"    ✗ Thumb error: {str(e)[:70]}")
    return False

# ─────────────────────────────────────────────────────────────────────────────
# METADATA
# ─────────────────────────────────────────────────────────────────────────────

metadata_records = []

def log_export(task_id, description, region_name,
               start_date, end_date, image_count, phase, export_type):
    metadata_records.append({
        'task_id':     task_id,
        'description': description,
        'region':      region_name,
        'start_date':  start_date,
        'end_date':    end_date,
        'image_count': image_count,
        'phase':       phase,
        'export_type': export_type,
        'timestamp':   datetime.now().isoformat(),
        'scale_m':     SCALE_METERS,
        'crs':         'EPSG:32644',
        'dtype':       'Float32',
        'bands':       ','.join(ALL_EXPORT_BANDS),
    })

def save_metadata_csv():
    if not metadata_records:
        return
    csv_path = f"{OUTPUT_DIR}/metadata/export_log.csv"
    with open(csv_path, 'w', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=metadata_records[0].keys())
        writer.writeheader()
        writer.writerows(metadata_records)
    print(f"✓ Metadata saved: {csv_path}  ({len(metadata_records)} records)")

def save_task_ids(tasks):
    task_file = f"{OUTPUT_DIR}/logs/task_ids.json"
    data = []
    for t in tasks:
        if t is None:
            continue
        try:
            data.append({
                'id':          t.id,
                'description': t.config.get('description', ''),
                'status':      t.status().get('state', 'UNKNOWN'),
            })
        except Exception:
            pass
    with open(task_file, 'w') as f:
        json.dump(data, f, indent=2)
    print(f"✓ Task IDs: {task_file}")
    print(f"  ▶  Monitor: https://code.earthengine.google.com/tasks")

# ─────────────────────────────────────────────────────────────────────────────
# DISPATCHER
# ─────────────────────────────────────────────────────────────────────────────

def dispatch_export(composite, desc, roi,
                    region_name, start_date, end_date,
                    count, phase, all_tasks, thumb_subdir='exports'):
    task = None
    try:
        if EXPORT_MODE == 'drive':
            task = export_to_drive(composite, desc, roi)
            all_tasks.append(task)

        elif EXPORT_MODE == 'gcs':
            if not GCS_BUCKET:
                raise ValueError("Set GCS_BUCKET for 'gcs' mode")
            task = export_to_gcs(composite, desc, roi, GCS_BUCKET)
            all_tasks.append(task)

        else:  # local_thumb
            thumb_path = f"{OUTPUT_DIR}/{thumb_subdir}/{desc}.png"
            if os.path.exists(thumb_path):
                return False
            if not download_local_thumb(composite, roi, thumb_path):
                return False

        task_id = task.id if task else 'local'
        log_export(task_id, desc, region_name,
                   start_date, end_date, count, phase, EXPORT_MODE)
        return True

    except Exception as e:
        print(f"    ✗ [{desc}]: {str(e)[:80]}")
        return False

# ─────────────────────────────────────────────────────────────────────────────
# MAIN
# ─────────────────────────────────────────────────────────────────────────────

def main():
    monthly_potential  = len(TIME_PERIODS) * len(REGIONS)
    annual_potential   = 13 * len(REGIONS)
    seasonal_potential = 12 * len(REGIONS) * 2
    total_potential    = monthly_potential + annual_potential + seasonal_potential

    print("=" * 72)
    print("  HASDEO GeoTIFF DOWNLOADER — FINAL FIXED (force_float32)")
    print("=" * 72)
    print(f"  Fix        : force_float32() — band-by-band explicit cast")
    print(f"  All bands  : {ALL_EXPORT_BANDS}")
    print(f"  Export     : {EXPORT_MODE.upper()}  |  {SCALE_METERS}m  |  EPSG:32644")
    print(f"  Regions    : {len(REGIONS)}  |  Periods: {len(TIME_PERIODS)}")
    print(f"  Target     : {total_potential:,} images")
    print("=" * 72)

    init_ee()
    create_dirs()

    all_tasks     = []
    export_count  = 0
    skipped_count = 0

    # ── PART 1: Monthly composites ────────────────────────────────────────
    print("\n[ PART 1 ]  Monthly composites — Jan 2013 to Aug 2025")
    print("-" * 72)

    for region_name, coords in REGIONS.items():
        roi = ee.Geometry.Rectangle(coords)
        print(f"\n  🌳 {region_name}")

        for start_date, end_date, phase in tqdm(TIME_PERIODS,
                                                desc=f"  {region_name}",
                                                unit='month'):
            try:
                col   = get_s2_collection(roi, start_date, end_date)
                count = col.size().getInfo()
                if count < MIN_IMAGES_PER_PERIOD:
                    skipped_count += 1
                    continue

                # build_composite() applies force_float32() internally
                composite  = build_composite(col, roi)
                year_month = start_date[:7].replace('-', '_')
                desc       = f"MON_{region_name}_{year_month}_{phase}"[:90]

                ok             = dispatch_export(composite, desc, roi,
                                                 region_name, start_date,
                                                 end_date, count, phase,
                                                 all_tasks)
                export_count  += ok
                skipped_count += (not ok)
                time.sleep(0.4)

            except Exception as e:
                print(f"\n    ✗ [{start_date}]: {str(e)[:80]}")

    # ── PART 2: Annual mosaics ────────────────────────────────────────────
    print("\n\n[ PART 2 ]  Annual mosaics — 2013–2025")
    print("-" * 72)

    for region_name, coords in REGIONS.items():
        roi = ee.Geometry.Rectangle(coords)
        print(f"\n  🌳 {region_name}")

        for year in range(2013, 2026):
            try:
                mosaic, count = create_annual_mosaic(roi, year)
                if count == 0:
                    skipped_count += 1
                    continue

                desc = f"ANN_{region_name}_{year}"[:90]
                ok   = dispatch_export(mosaic, desc, roi,
                                       region_name,
                                       f'{year}-01-01', f'{year}-12-31',
                                       count, 'annual_mosaic', all_tasks,
                                       thumb_subdir='annual')
                export_count  += ok
                skipped_count += (not ok)
                if ok:
                    print(f"    ✓ {year}: {count} scenes queued")
                time.sleep(0.4)

            except Exception as e:
                print(f"    ✗ {year}: {str(e)[:80]}")

    # ── PART 3: Seasonal composites ───────────────────────────────────────
    print("\n\n[ PART 3 ]  Seasonal composites — dry + wet — 2013–2024")
    print("-" * 72)

    for region_name, coords in REGIONS.items():
        roi = ee.Geometry.Rectangle(coords)
        print(f"\n  🌳 {region_name}")

        for year in range(2013, 2025):
            for season in ('dry', 'wet'):
                try:
                    composite, count = create_seasonal_composite(roi, year, season)
                    if count == 0:
                        skipped_count += 1
                        continue

                    s_date = (f'{year}-11-01' if season == 'dry'
                              else f'{year}-05-01')
                    e_date = (f'{year + 1}-04-30' if season == 'dry'
                              else f'{year}-10-31')
                    desc   = f"SEAS_{season.upper()}_{region_name}_{year}"[:90]

                    ok = dispatch_export(composite, desc, roi,
                                         region_name, s_date, e_date,
                                         count, f'{season}_composite',
                                         all_tasks,
                                         thumb_subdir='seasonal')
                    export_count  += ok
                    skipped_count += (not ok)
                    if ok:
                        print(f"    ✓ {season} {year}: {count} scenes")
                    time.sleep(0.4)

                except Exception as e:
                    print(f"    ✗ {season} {year}: {str(e)[:80]}")

    # ── SUMMARY ───────────────────────────────────────────────────────────
    print("\n" + "=" * 72)
    print("  FINAL SUMMARY")
    print("=" * 72)
    print(f"  ✓ Queued successfully : {export_count}")
    print(f"  ⊙ Skipped             : {skipped_count}")
    print(f"  📊 Potential total    : {total_potential:,}")
    print(f"  🔧 Type               : Float32 (all {len(ALL_EXPORT_BANDS)} bands)")
    print()

    if EXPORT_MODE in ('drive', 'gcs'):
        dest = ("Google Drive → 'Hasdeo_Training_Dataset/'"
                if EXPORT_MODE == 'drive'
                else f"gs://{GCS_BUCKET}/hasdeo/")
        print(f"  📁 {dest}")
        print("  🔗 Monitor: https://code.earthengine.google.com/tasks")
        save_task_ids(all_tasks)
    else:
        print(f"  📁 Local PNGs: {OUTPUT_DIR}/")

    save_metadata_csv()
    print("=" * 72)
    print("  ✅ All tasks queued — no more type mismatch errors.")


if __name__ == '__main__':
    main()

  HASDEO GeoTIFF DOWNLOADER — FINAL FIXED (force_float32)
  Fix        : force_float32() — band-by-band explicit cast
  All bands  : ['B2', 'B3', 'B4', 'B8', 'B8A', 'B11', 'B12', 'SCL', 'NDVI', 'NDWI', 'NBR', 'EVI']
  Export     : DRIVE  |  10m  |  EPSG:32644
  Regions    : 7  |  Periods: 152
  Target     : 1,323 images
✓ Earth Engine initialized | project: glacier-probe-model-475519
✓ Directories ready: hasdeo_training_dataset/

[ PART 1 ]  Monthly composites — Jan 2013 to Aug 2025
------------------------------------------------------------------------

  🌳 Hasdeo_Full


  Hasdeo_Full: 100%|███████████████████████| 152/152 [03:49<00:00,  1.51s/month]



  🌳 PEKB_Core


  PEKB_Core: 100%|█████████████████████████| 152/152 [03:22<00:00,  1.33s/month]



  🌳 Kente_Extension


...


  🌳 Hasdeo_South


  Hasdeo_South: 100%|██████████████████████| 152/152 [03:32<00:00,  1.40s/month]



  🌳 Mining_Buffer


  Mining_Buffer: 100%|█████████████████████| 152/152 [03:30<00:00,  1.38s/month]




[ PART 2 ]  Annual mosaics — 2013–2025
------------------------------------------------------------------------

  🌳 Hasdeo_Full
    ✓ 2016: 3 scenes queued
    ✓ 2017: 18 scenes queued
    ✓ 2018: 24 scenes queued
    ✓ 2019: 71 scenes queued
    ✓ 2020: 63 scenes queued
    ✓ 2021: 74 scenes queued
    ✓ 2022: 73 scenes queued
    ✓ 2023: 65 scenes queued
    ✓ 2024: 64 scenes queued
    ✓ 2025: 89 scenes queued

  🌳 PEKB_Core
    ✓ 2016: 2 scenes queued
    ✓ 2017: 9 scenes queued
    ✓ 2018: 15 scenes queued
    ✓ 2019: 35 scenes queued
    ✓ 2020: 31 scenes queued
    ✓ 2021: 36 scenes queued
    ✓ 2022: 38 scenes queued
    ✓ 2023: 32 scenes queued
    ✓ 2024: 32 scenes queued
    ✓ 2025: 44 scenes queued

  🌳 Kente_Extension
    ✓ 2016: 2 scenes queued
    ✓ 2017: 9 scenes queued
    ✓ 2018: 15 scenes queued
    ✓ 2019: 35 scenes queued
    ✓ 2020: 31 scenes queued
    ✓ 2021: 36 scenes queued
    ✓ 2022: 38 scenes queued
    ✓ 2023: 33 scenes queued
    ✓ 2024: 32 scenes queu